In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt

In [2]:
asset = 'TSLA'

In [4]:
intraday = yf.download(asset,start='2021-11-01', end='2021-11-02', interval='1m')

[*********************100%***********************]  1 of 1 completed


In [5]:
intraday

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-01 09:30:00-04:00,1145.000000,1150.000000,1140.930054,1149.989990,1149.989990,2916788
2021-11-01 09:31:00-04:00,1150.000000,1150.000000,1138.380005,1138.439941,1138.439941,361907
2021-11-01 09:32:00-04:00,1138.637207,1139.619995,1134.586060,1138.312012,1138.312012,359570
2021-11-01 09:33:00-04:00,1138.839966,1139.900024,1134.880005,1138.859985,1138.859985,258971
2021-11-01 09:34:00-04:00,1138.724976,1138.939941,1131.194946,1133.107666,1133.107666,357989
...,...,...,...,...,...,...
2021-11-01 15:55:00-04:00,1199.449951,1203.550049,1199.430054,1203.219971,1203.219971,411598
2021-11-01 15:56:00-04:00,1202.864990,1206.939941,1201.540039,1206.444946,1206.444946,346950
2021-11-01 15:57:00-04:00,1206.423950,1207.719971,1204.849976,1206.959961,1206.959961,384263


In [15]:
def Intradaytrend(df, entry, exit):
    ret_120min = df.iloc[120].Open/df.iloc[0].Open - 1
    tickret = df.Open.pct_change()
    if ret_120min > entry:
        buyprice = df.iloc[121].Open
        buytime = df.iloc[121].name
        cumulated = (tickret.loc[buytime:] + 1).cumprod() - 1
        exittime = cumulated[(cumulated < -exit) | (cumulated > exit)].first_valid_index()
        if exittime == None:
            exitprice = df.iloc[-1].Open
        else: 
            exitprice = df.loc[exittime + dt.timedelta(minutes=1)].Open
        profit = exitprice - buyprice
        profitrel = profit/buyprice
        return profitrel
    else: 
        return None

In [17]:
Intradaytrend(intraday, 0.01, 0.01)

0.011327892083682628

In [26]:
datesframe = yf.download(asset, start ='2021-11-01', end='2021-11-21')

[*********************100%***********************]  1 of 1 completed


In [27]:
datesframe.index

DatetimeIndex(['2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04',
               '2021-11-05', '2021-11-08', '2021-11-09', '2021-11-10',
               '2021-11-11', '2021-11-12', '2021-11-15', '2021-11-16',
               '2021-11-17', '2021-11-18', '2021-11-19'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [28]:
frames = []

for i in datesframe.index:
    frames.append(yf.download(asset, start=i, end= i + dt.timedelta(days=1),interval ='1m'))



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [41]:
returns = []

for i in frames:
    returns.append(Intradaytrend(i, 0.01, 0.02))

In [42]:
returns

[0.024681677634687555,
 -0.01604753423190458,
 None,
 None,
 None,
 -0.021035168078512332,
 None,
 -0.02446837950087822,
 None,
 None,
 None,
 0.009545712720914408,
 -0.019523950406091856,
 None,
 None]

In [43]:
pd.DataFrame(returns).mean()

0   -0.007808
dtype: float64